In [3]:
import json
import numpy as np
from textblob import TextBlob

userdict={}
with open('./yelp/user.json') as user_file:
    for line in user_file:
        d=json.loads(line)
        if (d["review_count"]>150):
            userdict[d["user_id"]] = d["review_count"] 

In [4]:
count={}
with open('./yelp/review.json') as review_file:
    for line in review_file:
        d=json.loads(line)        
        if (d["user_id"] in count):
            count[d["user_id"]]+=1
        else:
            count[d["user_id"]]=1

userlist=[]
for key in count:
    if (count[key]>100):
        userlist.append(key)


In [5]:
uid = userlist[0]
reviewdict={}
with open('./yelp/review.json') as review_file:
    for line in review_file:
        d=json.loads(line)
        if (d["user_id"]==uid):
            key=int(d["date"].replace("-", "")+"00")
            while (key in reviewdict):
                key+=1
            ls = [d["business_id"], d["stars"], d["text"]]
            reviewdict[key] = ls

In [6]:
dates = sorted(reviewdict, key=reviewdict.get)

In [7]:
n_stars=[]
n_sentiments= []
for i in range(30):
    n_stars.append(reviewdict[dates[i]][1])
    review_text = reviewdict[dates[i]][2]
    blob = TextBlob(review_text)
    n_sentiments.append(blob.sentiment.polarity)

    
print np.mean(n_stars)
print np.mean(n_sentiments)

3.76666666667
0.338488578576


In [8]:
#randomly choose 30 reviews for prediction
import random
dates2 = dates[30:]
random.shuffle(dates2)
dates2 = dates2[:30]
print dates2

[2014092900, 2011101400, 2014081200, 2011072400, 2011051303, 2010050800, 2011052301, 2011051302, 2011031901, 2012032800, 2010040300, 2011060300, 2011111304, 2011060303, 2010061301, 2012081900, 2014011401, 2011100705, 2011010101, 2014042207, 2013091800, 2013121105, 2010030600, 2010052201, 2011112800, 2010042500, 2012021200, 2010050802, 2011072901, 2011111303]


In [9]:
businessdict = {}
with open('./yelp/business.json') as business_file:
    for line in business_file:
        d=json.loads(line)
        businessdict[d["business_id"]] = d["stars"]
print len(businessdict)

61184


In [10]:
#get the sentiment of the current review and the rate
actual_stars=[]
pred_sentiments=[]
business_stars=[]
for date in dates2:
    business_stars.append(businessdict[reviewdict[date][0]])
    actual_stars.append(reviewdict[date][1])
    pred_text = reviewdict[date][2]
    blob = TextBlob(pred_text)
    pred_sentiments.append(blob.sentiment.polarity)
print business_stars
print actual_stars
print pred_sentiments



[3.5, 4.5, 4.0, 4.0, 3.5, 4.0, 4.0, 3.5, 4.0, 4.5, 3.5, 4.5, 4.0, 4.5, 4.0, 4.0, 4.5, 3.5, 4.5, 3.5, 4.0, 4.5, 4.0, 3.5, 4.0, 4.5, 4.5, 3.5, 4.0, 3.5]
[2, 4, 4, 3, 4, 4, 4, 4, 5, 4, 4, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 5, 4, 4, 5, 3, 5, 3, 4, 3]
[-0.08333333333333334, 0.2913469106650925, 0.190625, 0.5166666666666666, 0.30666666666666675, 0.3374868247694335, 0.1330357142857143, 0.24903846153846157, 0.2890625, 0.4708333333333334, 0.178125, 0.14375, 0.34687499999999993, 0.34845679012345676, 0.40285714285714286, 0.3687500000000001, 0.5333333333333333, -0.008928571428571432, 0.3507954545454546, 0.47880952380952374, 0.3199285714285714, 0.42750000000000005, 0.3372727272727273, 0.016666666666666653, 0.4383653846153846, 0.22366071428571432, 0.4666666666666666, 0.36178571428571427, 0.3, 0.13999999999999999]


In [11]:
rate=[]
for i in range(30):
    rate.append(n_sentiments[i]/n_stars[i])
avg_rate= np.mean(rate)
print avg_rate

0.0859613782479


In [14]:
test_rate=[]
for i in range(30):
    test_rate.append(pred_sentiments[i]/actual_stars[i]-avg_rate)


In [18]:
#calculate the predicted rating based on users average sentiment
pred_stars=[]
avg_stars=np.mean(n_stars)
avg_sentiments = np.mean(n_sentiments)
for i in range(30):
    pred=(pred_sentiments[i]-avg_sentiments)/2*5+avg_stars
    pred=min(5, pred)
    pred=max(1, pred)
    pred_stars.append(pred)
print pred_stars
print actual_stars
print business_stars
    
        


[2.7121118868929917, 3.6488124968890565, 3.3970077202263251, 4.2121118868929921, 3.6871118868929922, 3.764162282149909, 3.2530345059406107, 3.5430413740724793, 3.6431014702263251, 4.0975285535596591, 3.3657577202263251, 3.2798202202263251, 3.7876327202263251, 3.7915871955349671, 3.9275880773691823, 3.8423202202263256, 4.2537785535596582, 2.8981237916548963, 3.7974338565899615, 4.1174690297501346, 3.7202666487977538, 3.9891952202263252, 3.7636270384081434, 2.9621118868929921, 4.0163586817647863, 3.4795970059406112, 4.0871118868929921, 3.8249095059406111, 3.6704452202263251, 3.2704452202263252]
[2, 4, 4, 3, 4, 4, 4, 4, 5, 4, 4, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 5, 4, 4, 5, 3, 5, 3, 4, 3]
[3.5, 4.5, 4.0, 4.0, 3.5, 4.0, 4.0, 3.5, 4.0, 4.5, 3.5, 4.5, 4.0, 4.5, 4.0, 4.0, 4.5, 3.5, 4.5, 3.5, 4.0, 4.5, 4.0, 3.5, 4.0, 4.5, 4.5, 3.5, 4.0, 3.5]


In [22]:
count=0
for i in range(30):
    if (business_stars[i]-pred_stars[i])*(actual_stars[i]-pred_stars[i])>0:
        count+=1

print count/30.0

0.666666666667


In [ ]:
from textblob import TextBlob

text = ""
blob = TextBlob(text)
print blob.sentiment.polarity
print blob.sentiment.subjectivity
for sentence in blob.sentences:
   print sentence.sentiment.polarity
   print sentence.sentiment.subjectivity
    